# GRU (Gated Recurrent Unit)


In [1]:
import numpy as np
from keras.callbacks import (
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)
from keras.layers import GRU, Dense, Input
from keras.losses import MeanSquaredError
from keras.metrics import MeanSquaredError as MSEMetric
from keras.models import Sequential
from keras.optimizers import Adam

In [2]:
# The Fibonacci Sequence
def fibonacci(n):
    """Generate Fibonacci sequence up to n terms."""
    fib_list = [0, 1]
    for i in range(2, n):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list[:n]


raw_seq = fibonacci(15)
n_steps = 5  # Use 5 previous numbers to predict the next
n_features = 1


# Function to convert a sequence into (X, y) samples
def create_sequences(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


X, y = create_sequences(raw_seq, n_steps)

# Reshape input to be [samples, timesteps, features] for LSTM
X = X.reshape((X.shape[0], X.shape[1], n_features))

print(f"Sample X (Input): {X[0].flatten()} -> y (Output): {y[0]}")
print(f"Sample X (Input): {X[-1].flatten()} -> y (Output): {y[-1]}")

Sample X (Input): [0 1 1 2 3] -> y (Output): 5
Sample X (Input): [ 34  55  89 144 233] -> y (Output): 377


In [3]:
gru = Sequential(
    layers=[
        Input(shape=(n_steps, n_features)),
        GRU(64),
        Dense(64),
        Dense(32),
        Dense(1),
    ],
    name="gru",
)

In [4]:
gru.layers

[<GRU name=gru, built=True>,
 <Dense name=dense, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>]

In [5]:
gru.summary()

Model: "gru"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,137 (74.75 KB)

 Trainable params: 19,137 (74.75 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
gru.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MSEMetric()])  # type: ignore

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor="loss", patience=10, restore_best_weights=True
)
model_checkpoint_callback = ModelCheckpoint(
    filepath="../../Models/gru.keras",
    monitor="loss",
    save_best_only=True,
    mode="min",
)
tensorboard_callback = TensorBoard(
    log_dir="../../Logs/gru_logs",
    histogram_freq=1,
    write_images=True,
    write_steps_per_second=True,
)
# tensorboard --logdir="Logs/gru_logs"

csvlogger_callback = CSVLogger("../../Logs/gru_logs.csv")

reduce_lr_callback = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=5)

gru_history = gru.fit(
    X,
    y,
    epochs=200,
    callbacks=[
        early_stopping_callback,
        model_checkpoint_callback,
        tensorboard_callback,
        csvlogger_callback,
        reduce_lr_callback,
    ],
)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 22828.8262 - mean_squared_error: 22828.8262 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step - loss: 22686.0820 - mean_squared_error: 22686.0820 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 590ms/step - loss: 22544.2910 - mean_squared_error: 22544.2910 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - loss: 22402.5449 - mean_squared_error: 22402.5449 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 22260.2617 - mean_squared_error: 22260.2617 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 22116.9102 - mean_squared_error: 22116.9102 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 21971.9414 - mean_squared_error: 21971.9414 - learning_rate: 0.0010
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 21824.8398 - mean_squared_error: 21824.8398 - learning_

In [8]:
# New input data (the last n_steps numbers)
x_input = np.array([55, 89, 144, 233, 377])

# Reshape the input for the model: (1 sample, n_steps timesteps, 1 feature)
x_input = x_input.reshape((1, n_steps, n_features))

# Make the prediction
yhat = gru.predict(x_input)

print(f"\n--- Prediction Result ---")
print(f"Input Sequence: {x_input.flatten()}")
# Round the prediction as Fibonacci numbers are integers
print(f"Predicted Next Number: {round(yhat[0][0])}")
print(f"Expected Next Number: ", x_input.flatten()[-1] + x_input.flatten()[-2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step

--- Prediction Result ---
Input Sequence: [ 55  89 144 233 377]
Predicted Next Number: 395
Expected Next Number:  610
